
### Part 1 Extract, transform, load, snow pit and transect data
[Josh King](https://github.com/kingjml), *CPS/CRD/ECCC*, 2021  
[Benoit Montpetit](https://github.com/ecccben), *CPS/CRD/ECCC*, 2024  
[Mike Brady](https://github.com/m9brady), *CPS/CRD/ECCC*, 2024

This notebook provides data extraction, transformation and loading procedures for field data collected during the 2018-2019 TVCSnow campaign. The full dataset contains measurements which exceed the discussed material within this paper (e.g. 3 observation periods, where as only 1 is discussed in text).

Data at all sites (both static and roving) comes from 4 sources which are imported and prepared for analysis. We create reference SSA and density profiles from the imported snow pit and IceCube data. All data are UTM8N georefereced (EPSG 32608). Finalized datasets are exported to be used in further analysis steps. Second, we generate products to represent the snowpack at each site for further analysis and modeling. Imported snow pits are in the standard ECCC format and are read with the [snow pit parser](https://github.com/kingjml/tvc-snowpit-parser) developed along with [Mike Brady](https://github.com/m9brady) and [Ben Montpetit](https://github.com/ECCCBen).


Output of the workbook are science ready snow products for the TVC domain in pickle format.


In [ ]:
# Establish path from notebook
import sys
sys.path.append("..")

# Community imports
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.mask
from shapely.geometry import mapping
from shapely.wkt import loads as wkt_load

# Plot imports and settings
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
import matplotlib

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

# ECCC imports
import constants
import tvcfunc
from snowpit_datasheet_parser import SnowPitSheet

# Paths to external files and constants
ext_file_list = [constants.VEG_TYPE_FILE, constants.VEG_HEIGHT_FILE, constants.DTM_ELV_FILE]
ext_file_var = ['veg_type', 'veg_height', 'dtm_elv', 'dtm_slope']
site_dirs = Path('../Data/Site')

## Crawl site directories and match data sources

This section crawls our field data and generates metadata. Input has been structured into site folders. The following datasets are evaluated:

1. **mp_files** - Magnaprobe Snow Depth - ECCC Standardized Excel Format
2. **pit_files** - Standard Snow Pits - ECCC Standardized Excel Format
3. **ssa_files** - IceCube SSA - ECCC Standardized Excel Format
4. **pnt_files** -  SnowMicroPen Penetration Force - SLF PNT Format

Output is a validated list of field datasets available for each site.

In [ ]:
site_meta = tvcfunc.gen_site_meta(site_dirs)
site_meta.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site</th>
      <th>pnt_files</th>
      <th>smp_meta</th>
      <th>pit_files</th>
      <th>ssa_files</th>
      <th>mp_files</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>RS01</td>
      <td>[Data\Site\RS01\S34M2527.pnt, Data\Site\RS01\S...</td>
      <td>[Data\Site\RS01\SMP_131118_RS01.csv]</td>
      <td>[Data\Site\RS01\PIT_131118_RS01.xlsx]</td>
      <td>[Data\Site\RS01\SSA_131118_RS01.csv]</td>
      <td>[Data\Site\RS01\MP_131118_RS01.xlsx]</td>
    </tr>
    <tr>
      <th>1</th>
      <td>RS02</td>
      <td>[Data\Site\RS02\S34M2546.pnt, Data\Site\RS02\S...</td>
      <td>[Data\Site\RS02\SMP_131118_RS02.csv]</td>
      <td>[Data\Site\RS02\PIT_131118_RS02.xlsx]</td>
      <td>[Data\Site\RS02\SSA_131118_RS02.csv]</td>
      <td>[Data\Site\RS02\MP_131118_RS02.xlsx]</td>
    </tr>
    <tr>
      <th>2</th>
      <td>RS03</td>
      <td>[Data\Site\RS03\S34M2622.pnt, Data\Site\RS03\S...</td>
      <td>[Data\Site\RS03\SMP_141118_RS03.csv]</td>
      <td>[Data\Site\RS03\PIT_141118_RS03.xlsx]</td>
      <td>[Data\Site\RS03\SSA_141118_RS03.csv]</td>
      <td>[Data\Site\RS03\MP_141118_RS03.xlsx]</td>
    </tr>
    <tr>
      <th>3</th>
      <td>RS04</td>
      <td>[Data\Site\RS04\S34M2640.pnt, Data\Site\RS04\S...</td>
      <td>[Data\Site\RS04\SMP_141118_RS04.csv]</td>
      <td>[Data\Site\RS04\PIT_141118_RS04.xlsx]</td>
      <td>[Data\Site\RS04\SSA_141118_RS04.csv]</td>
      <td>[Data\Site\RS04\MP_141118_RP04.xlsx]</td>
    </tr>
    <tr>
      <th>4</th>
      <td>RS05</td>
      <td>[Data\Site\RS05\S34M2660.pnt, Data\Site\RS05\S...</td>
      <td>[Data\Site\RS05\SMP_151118_RS05.csv]</td>
      <td>[Data\Site\RS05\PIT_151118_RS05.xlsx]</td>
      <td>[Data\Site\RS05\SSA_151118_RS05.csv]</td>
      <td>[Data\Site\RS05\MP_151118_RS05.xlsx]</td>
    </tr>
  </tbody>
</table>
</div>

# ETL snow pit reference data
This section loads all of the snow pit sheets and pulls meta data to create geospatial dataframe projected in UTM8N.

I then extract the density, ssa, and temperature profiles from each snow pit to generate and validate reference datasets. Each profile includes the associated grain type as well as relative height from the air-snow interface. Temperature profiles include tagging for the media type of the measurement.

I use the [SnowPitSheet library](https://github.com/kingjml/tvc-snowpit-parser) to read in our ECCC CPS style pit sheets. QA level metadata are provided in pit_meta.

In [ ]:
# Load the snow pit data and create a geospatial dataframe
pit_files = site_meta['pit_files'].map(lambda x: x[0]).values
pit_data = [SnowPitSheet(pit) for pit in pit_files]
pit_df = [pd.DataFrame.from_dict(pit.meta, orient='index').transpose() for pit in pit_data]
pit_df = pd.concat(pit_df, axis=0)
pit_df['file_path'] = pit_files
pit_gdf = gpd.GeoDataFrame(data=pit_df.drop('geometry', axis=1), 
                           geometry=pit_df['geometry'].apply(wkt_load), 
                           crs=constants.CRS_WGS84)

pit_meta = pit_gdf.to_crs(constants.CRS_UTM8N).reset_index(drop=True)
pit_meta['timestamp'] = pd.to_datetime(pit_meta['timestamp']) 
pit_meta['site'] =pit_meta['pit_id'].str.replace('RP', 'RS', regex=True)

# Tag the campaign
pit_meta.loc[pit_meta['site'].isin(constants.TVC01), 'campaign'] = 'TVC01'
pit_meta.loc[pit_meta['site'].isin(constants.TVC02), 'campaign'] = 'TVC02'
pit_meta.loc[pit_meta['site'].isin(constants.TVC03), 'campaign'] = 'TVC03'
pit_meta.head(2)

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>location</th>
      <th>timestamp</th>
      <th>surveyors</th>
      <th>site</th>
      <th>pit_id</th>
      <th>slope</th>
      <th>total_depth</th>
      <th>utm_zone</th>
      <th>comments</th>
      <th>weather</th>
      <th>...</th>
      <th>wind_1</th>
      <th>wind_2</th>
      <th>ground_condition</th>
      <th>soil_moisture</th>
      <th>ground_roughness</th>
      <th>ground_vegetation</th>
      <th>tree_canopy</th>
      <th>file_path</th>
      <th>geometry</th>
      <th>campaign</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>TVC 2018</td>
      <td>2018-11-13 09:48:00</td>
      <td>Josh, Ben</td>
      <td>RS01</td>
      <td>RP01</td>
      <td>Flat</td>
      <td>34.5</td>
      <td>8N</td>
      <td>Comments/Notes/Indicate co-located measurements:</td>
      <td>First roving site. Located at site waypoint #....</td>
      <td>...</td>
      <td>None</td>
      <td>None</td>
      <td>None</td>
      <td>None</td>
      <td>None</td>
      <td>{}</td>
      <td>None</td>
      <td>Data\Site\RS01\PIT_131118_RS01.xlsx</td>
      <td>POINT (561402.426 7627347.290)</td>
      <td>TVC01</td>
    </tr>
    <tr>
      <th>1</th>
      <td>TVC 2018</td>
      <td>2018-11-13 11:53:00</td>
      <td>Josh, Ben</td>
      <td>RS02</td>
      <td>RP02</td>
      <td>Flat</td>
      <td>28.0</td>
      <td>8N</td>
      <td>Comments/Notes/Indicate co-located measurements:</td>
      <td>Located on a plateau. AT = -23, winds at 3 m/s...</td>
      <td>...</td>
      <td>None</td>
      <td>None</td>
      <td>None</td>
      <td>None</td>
      <td>None</td>
      <td>{}</td>
      <td>None</td>
      <td>Data\Site\RS02\PIT_131118_RS02.xlsx</td>
      <td>POINT (560862.323 7627725.511)</td>
      <td>TVC01</td>
    </tr>
  </tbody>
</table>
<p>2 rows × 22 columns</p>
</div>

In [ ]:
# Create reference density, SSA and temperature  profiles for each site
ref_rho = pd.DataFrame(); ref_ssa = pd.DataFrame(); ref_temp = pd.DataFrame()
for s_dix, s_meta in site_meta.iterrows():  
    pit_path = s_meta['pit_files'][0] # Only 1 pit per site for TVC
    ssa_path = s_meta['ssa_files'][0] # TODO: deal with multiple SSA files
    ref_rho = pd.concat([ref_rho,pd.DataFrame(tvcfunc.load_ref_rho(pit_path).assign(site = s_meta['site']))], ignore_index = True)
    ref_ssa = pd.concat([ref_ssa,pd.DataFrame(tvcfunc.load_ref_ssa(ssa_path, pit_path).assign(site = s_meta['site']))], ignore_index = True)
    ref_temp = pd.concat([ref_temp,pd.DataFrame(tvcfunc.load_ref_temperature(pit_path).assign(site = s_meta['site']))], ignore_index = True)

# Example snow pit data
The next two plots show observed density and SSA at (1) a single location and (2) for all snow pits

In [ ]:
# Example snowpit of reference information
site_select = 'SM02'
dens = ref_rho.loc[ref_rho['site'] == site_select]
ssa = ref_ssa.loc[ref_ssa['site'] == site_select]
fig = plt.figure(figsize=(10, 10))
ax1 = fig.add_subplot(1,1,1)
ax2 = ax1.twiny()
lns1 = ax1.plot(ssa['ssa'], ssa['height']/1000, marker = 'o', color = 'k', label = 'SSA')
lns2 = ax2.plot(dens['density'], dens['height']/1000, marker = 'o', color = 'teal', label = 'Density')
ax1.set_xlim(0,ssa['ssa'].max()+5)
ax1.set_ylim(0, ssa['height'].max()/1000 + 0.055)
ax2.set_xlim(75,dens['height'].max())
ax1.set_xlabel('SSA $m^2\\cdot kg^{-1}$')
ax1.set_ylabel('Height above ground (mm)')
ax2.set_xlabel('$\\rho_{snow} (kg\\cdot m^{-3})$')

# added these three lines
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc=4)

<center><img src="../Figures/Part_1_ETL_Field_Fig1.png" height="500px"></center>

<center>Figure: Example of a snow density and SSA profile</center>

In [ ]:
# Limit data to only January campaign
ref_rho_tvc2 = ref_rho[ref_rho['site'].isin(constants.TVC02)]
ref_ssa_tvc2 = ref_ssa[ref_ssa['site'].isin(constants.TVC02)]

# Layer colours based on international classification of snow
colors = {'N':'#969696', 'F':'#ADD8E6', 'H':'#0000FF', 'R':'#FFB6C1', 'M':'grey'}

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15,10))
ax1.scatter(ref_rho_tvc2['density'],ref_rho_tvc2['height']/1000, s = 10, alpha = 1, c=ref_rho_tvc2['grain_type'].map(colors))
ax2.scatter(ref_ssa_tvc2['ssa'],ref_ssa_tvc2['height']/1000, s = 10, alpha = 1, c=ref_ssa_tvc2['grain_type'].map(colors))
ax2.set_xlim(0, 70)
ax2.set_ylim(0, 0.75)
ax2.set_xlabel('SSA [m$\mathregular{^2}$ kg$\mathregular{^{-1}}$]')
ax1.set_ylabel('Height above surface [m]')
ax1.set_xlabel('Density [kg m$\mathregular{^{-3}}$]')
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Rounded',
                          markerfacecolor='#FFB6C1', markersize=10),
                  Line2D([0], [0], marker='o', color='w', label='Faceted',
                          markerfacecolor='#ADD8E6', markersize=10),
                  Line2D([0], [0], marker='o', color='w', label='Hoar',
                          markerfacecolor='#0000FF', markersize=10),
                  Line2D([0], [0], marker='o', color='w', label='Mixed',
                          markerfacecolor='grey', markersize=10)]
ax2.legend(handles=legend_elements, fontsize=12)



<center><img src="../Figures/Part_1_ETL_Field_Fig2.png" height="500px"></center>

<center>Figure: Example of a snow density and SSA profile color coded by grain type</center>

In [ ]:
ref_ssa_tvc2.groupby('grain_type')['ssa'].describe()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>count</th>
      <th>mean</th>
      <th>std</th>
      <th>min</th>
      <th>25%</th>
      <th>50%</th>
      <th>75%</th>
      <th>max</th>
    </tr>
    <tr>
      <th>grain_type</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>F</th>
      <td>59.0</td>
      <td>25.180908</td>
      <td>8.250971</td>
      <td>12.911707</td>
      <td>18.891815</td>
      <td>23.762626</td>
      <td>28.141087</td>
      <td>46.787239</td>
    </tr>
    <tr>
      <th>H</th>
      <td>133.0</td>
      <td>13.374954</td>
      <td>2.221366</td>
      <td>9.901494</td>
      <td>12.098719</td>
      <td>13.067053</td>
      <td>13.944551</td>
      <td>24.251712</td>
    </tr>
    <tr>
      <th>M</th>
      <td>54.0</td>
      <td>40.384020</td>
      <td>13.607262</td>
      <td>13.037687</td>
      <td>30.130222</td>
      <td>36.398524</td>
      <td>51.564161</td>
      <td>72.452065</td>
    </tr>
    <tr>
      <th>R</th>
      <td>2.0</td>
      <td>45.078542</td>
      <td>3.012478</td>
      <td>42.948398</td>
      <td>44.013470</td>
      <td>45.078542</td>
      <td>46.143613</td>
      <td>47.208685</td>
    </tr>
  </tbody>
</table>
</div>

In [ ]:
ref_rho_tvc2.groupby('grain_type')['density'].describe()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>count</th>
      <th>mean</th>
      <th>std</th>
      <th>min</th>
      <th>25%</th>
      <th>50%</th>
      <th>75%</th>
      <th>max</th>
    </tr>
    <tr>
      <th>grain_type</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>F</th>
      <td>58.0</td>
      <td>301.413788</td>
      <td>65.989906</td>
      <td>153.0</td>
      <td>246.00</td>
      <td>299.0</td>
      <td>364.25</td>
      <td>410.0</td>
    </tr>
    <tr>
      <th>H</th>
      <td>128.0</td>
      <td>220.632812</td>
      <td>29.218931</td>
      <td>166.0</td>
      <td>208.75</td>
      <td>218.0</td>
      <td>229.00</td>
      <td>369.0</td>
    </tr>
    <tr>
      <th>M</th>
      <td>57.0</td>
      <td>325.333344</td>
      <td>81.980904</td>
      <td>116.0</td>
      <td>265.00</td>
      <td>334.0</td>
      <td>403.00</td>
      <td>438.0</td>
    </tr>
    <tr>
      <th>R</th>
      <td>2.0</td>
      <td>228.000000</td>
      <td>1.414214</td>
      <td>227.0</td>
      <td>227.50</td>
      <td>228.0</td>
      <td>228.50</td>
      <td>229.0</td>
    </tr>
  </tbody>
</table>
</div>

# Magnaprobe snow depth data
ETL for ECCC standard magnaprobe files to a generate geospatial dataframe

In [ ]:
# Load the magnaprobe data and create a geospatial dataframe
mp_data = pd.DataFrame()
mp_files = site_meta['mp_files'].map(lambda x: x[0]).values
mp_data = [pd.read_excel(mp).assign(site = site) for site, mp in zip(site_meta['site'].values, mp_files)]
mp_data = pd.concat(mp_data, axis=0)
mp_data = mp_data.drop(columns=['Counter','Unit'])
mp_data.columns = ['timestamp', 'depth', 'lat', 'lon', 'site']
mp_data = mp_data.loc[(mp_data['depth'] > 1 )| (mp_data['depth'] < 119)] # Drop marker measurements

mp_gdf = gpd.GeoDataFrame(mp_data, 
                          geometry=gpd.points_from_xy(mp_data.lon, mp_data.lat), 
                          crs = constants.CRS_WGS84)
mp_utm = mp_gdf.to_crs(constants.CRS_UTM8N)

mp_utm.loc[mp_utm['site'].isin(constants.TVC01), 'campaign'] = 'TVC01'
mp_utm.loc[mp_utm['site'].isin(constants.TVC02), 'campaign'] = 'TVC02'
mp_utm.loc[mp_utm['site'].isin(constants.TVC03), 'campaign'] = 'TVC03'

# This flags instances where the Magnaprobe RTC battery failed
time_fix, rtc_fail = tvcfunc.rtc_check_mp(mp_utm['timestamp'])
mp_utm = mp_utm.assign(timestamp = time_fix, rtc_fail = rtc_fail)

The next two plots show (1) the seasonal distrobutions of snows depth across all sites and (2) locations of the snow depths seperated by campaign.

In [ ]:
bins = np.arange(0,120, 2)
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(15,10))
ax1.hist(mp_utm.loc[mp_utm['campaign'] == 'TVC01']['depth'].values, 
         density=True, bins=bins, label = 'TVC-1', color = 'GREY')
ax2.hist(mp_utm.loc[mp_utm['campaign'] == 'TVC02']['depth'].values, 
         density=True, bins=bins, label = 'TVC-2', color = 'GREY') 
ax3.hist(mp_utm.loc[mp_utm['campaign'] == 'TVC03']['depth'].values, 
         density=True, bins=bins, label = 'TVC-3', color = 'GREY') 

ax1.set_title('TVC1 - Nov 2018')
ax2.set_title('TVC2 - Jan 2019')
ax3.set_title('TVC3 - Mar 2019')
ax2.set_xlabel('Snow Depth [cm]')
ax1.set_ylabel('Probability [%]')


mean_1 = mp_utm.loc[mp_utm['campaign'] == 'TVC01']['depth'].mean()
mean_2 = mp_utm.loc[mp_utm['campaign'] == 'TVC02']['depth'].mean()
mean_3 = mp_utm.loc[mp_utm['campaign'] == 'TVC03']['depth'].mean()

ax1.axvline(x=mean_1, color='r', linestyle='--')
ax2.axvline(x=mean_2, color='r', linestyle='--')
ax3.axvline(x=mean_3, color='r', linestyle='--')

ax1.set_xlim(0,100)
ax2.set_xlim(0,100)
ax3.set_xlim(0,100)

#f.savefig('Figures/site_magnaprobe_depth.jpg', dpi = 300)

mp_utm.groupby('campaign')['depth'].describe()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>count</th>
      <th>mean</th>
      <th>std</th>
      <th>min</th>
      <th>25%</th>
      <th>50%</th>
      <th>75%</th>
      <th>max</th>
    </tr>
    <tr>
      <th>campaign</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>TVC01</th>
      <td>6185.0</td>
      <td>34.019688</td>
      <td>9.535905</td>
      <td>0.007</td>
      <td>28.080</td>
      <td>33.15</td>
      <td>38.8500</td>
      <td>120.2</td>
    </tr>
    <tr>
      <th>TVC02</th>
      <td>6708.0</td>
      <td>46.696394</td>
      <td>13.213402</td>
      <td>12.950</td>
      <td>38.010</td>
      <td>45.61</td>
      <td>54.2125</td>
      <td>115.4</td>
    </tr>
    <tr>
      <th>TVC03</th>
      <td>8331.0</td>
      <td>43.795541</td>
      <td>14.033336</td>
      <td>8.250</td>
      <td>33.995</td>
      <td>42.48</td>
      <td>51.8800</td>
      <td>119.5</td>
    </tr>
  </tbody>
</table>
</div>

<center><img src="../Figures/Part_1_ETL_Field_Fig3.png" height="500px"></center>

<center>Figure: Snow depth distributions for each of the intensive campaigns (November, January and March) for the 2018/19 winter season</center>

# Compute summary site statistics
We match sites with the magnaprobe data to compute summary statistics. Distance between the magnaprobe points and the snow pit are also calculated to make sure we are on target. We then take the reference SSA, density, and temperature datasets to generate layer weighted averages.

In [ ]:
snow_stats = pd.DataFrame()
for idx, pit in pit_meta.iterrows():
    mp_local = mp_utm.loc[mp_utm['site'] == pit['site']]
    pit_local = gpd.GeoSeries(pit.geometry, crs = constants.CRS_UTM8N)
    dist_to_pit = mp_local.geometry.apply(lambda g: pit_local.distance(g))
    
    snow_stats = pd.concat([snow_stats,pd.DataFrame({'site': [pit['site']],
                  'depth_count': [mp_local['depth'].count()],
                  'depth_mean': [mp_local['depth'].mean()], 
                  'depth_median': [mp_local['depth'].median()],
                  'depth_std': [mp_local['depth'].std()],
                  'dist_max': [dist_to_pit.max()[0]],
                  'dist_min': [dist_to_pit.min()[0]],
                  'dist_mean': [dist_to_pit.mean()[0]]})],ignore_index=True)

# Get weighted mean of each site density ssa and temperature profile
for site in snow_stats['site'].unique():
    l_ssa = ref_ssa.loc[ref_ssa['site'] == site]['ssa'].values
    l_thickness_ssa = np.abs(np.diff(np.append(ref_ssa.loc[ref_ssa['site'] == site]['height'].values, 0)))
    snow_stats.loc[snow_stats['site'] == site, 'ssa_mean'] = np.average(l_ssa, weights = l_thickness_ssa)
    snow_stats.loc[snow_stats['site'] == site, 'ssa_count'] = len(l_thickness_ssa)
    
    l_dens = ref_rho.loc[ref_rho['site'] == site]['density'].values
    l_thickness_dens = np.abs(np.diff(np.append(ref_rho.loc[ref_rho['site'] == site]['height'].values, 0)))
    snow_stats.loc[snow_stats['site'] == site, 'dens_mean'] = np.average(l_dens, weights = l_thickness_dens)
    snow_stats.loc[snow_stats['site'] == site, 'dens_count'] = len(l_thickness_dens)
    
    # Limit to measurements in snow only
    ref_temp_snow = ref_temp.loc[(ref_temp['site'] == site) & (ref_temp['type'] == 'snow')]
    l_temp = ref_temp_snow['temperature_k'].values
    l_thickness_temp = np.abs(np.diff(np.append(ref_temp_snow['height'].values, 0)))
    snow_stats.loc[snow_stats['site'] == site, 'temp_mean'] = np.average(l_temp, weights = l_thickness_temp)
snow_stats['swe_mean'] = snow_stats['depth_mean']/100 * snow_stats['dens_mean']

snow_stats.head(5)

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>site</th>
      <th>depth_count</th>
      <th>depth_mean</th>
      <th>depth_median</th>
      <th>depth_std</th>
      <th>dist_max</th>
      <th>dist_min</th>
      <th>dist_mean</th>
      <th>ssa_mean</th>
      <th>ssa_count</th>
      <th>dens_mean</th>
      <th>dens_count</th>
      <th>temp_mean</th>
      <th>swe_mean</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>RS01</td>
      <td>348</td>
      <td>34.703563</td>
      <td>33.765</td>
      <td>7.216446</td>
      <td>228.342479</td>
      <td>4.151556</td>
      <td>83.327945</td>
      <td>19.821297</td>
      <td>6.0</td>
      <td>198.923077</td>
      <td>10.0</td>
      <td>258.716660</td>
      <td>69.033396</td>
    </tr>
    <tr>
      <th>1</th>
      <td>RS02</td>
      <td>288</td>
      <td>35.326840</td>
      <td>34.390</td>
      <td>8.756437</td>
      <td>157.222222</td>
      <td>1.597891</td>
      <td>68.399205</td>
      <td>17.792863</td>
      <td>5.0</td>
      <td>171.207547</td>
      <td>8.0</td>
      <td>259.609988</td>
      <td>60.482217</td>
    </tr>
    <tr>
      <th>2</th>
      <td>RS03</td>
      <td>249</td>
      <td>35.454819</td>
      <td>34.530</td>
      <td>8.763020</td>
      <td>162.440222</td>
      <td>2.861935</td>
      <td>71.542305</td>
      <td>21.571987</td>
      <td>7.0</td>
      <td>206.333333</td>
      <td>9.0</td>
      <td>262.199997</td>
      <td>73.155110</td>
    </tr>
    <tr>
      <th>3</th>
      <td>RS04</td>
      <td>249</td>
      <td>32.437510</td>
      <td>31.420</td>
      <td>7.040102</td>
      <td>182.928065</td>
      <td>0.983085</td>
      <td>78.184670</td>
      <td>23.773703</td>
      <td>6.0</td>
      <td>189.945455</td>
      <td>8.0</td>
      <td>260.189999</td>
      <td>61.613576</td>
    </tr>
    <tr>
      <th>4</th>
      <td>RS05</td>
      <td>280</td>
      <td>36.698964</td>
      <td>36.095</td>
      <td>8.458829</td>
      <td>172.568529</td>
      <td>2.032063</td>
      <td>70.086874</td>
      <td>18.298919</td>
      <td>6.0</td>
      <td>195.764706</td>
      <td>8.0</td>
      <td>258.889999</td>
      <td>71.843619</td>
    </tr>
  </tbody>
</table>
</div>

# Extract information from external datasets
For each site we extract terrain and vegetation information at each site from:

- [Grünberg and Boike (2019). Vegetation map of Trail Valley Creek](https://doi.pangaea.de/10.1594/PANGAEA.904270)
- [Anders, et al. (2018). Airborne Laser Scanning (ALS) Point Clouds of Trail Valley Creek)](https://doi.pangaea.de/10.1594/PANGAEA.894884)

The resulting dataset contains numpy masked arrays of veg height, veg type, and surface elevation for all grid cells within 50 m of all sites. 

Then generate summary statistics for each site with output variables (*aux_stats*):

- **site**: ECCC site names
- **pit_ts**: Timestamp of snow pit
- **veg_c_per**: Percentage coverage of each veg class in constants.VEG_CLASS
- **veg_h_mean**: Mean veg height
- **veg_h_std**: Standard dev of veg height
- **dem_h_mean**: Mean DEM height
- **dem_h_std**: Standard dev of DEM height

In [ ]:
pit_ext = pd.concat([pit_meta,pd.DataFrame(columns=ext_file_var)])
for site_idx, site in pit_meta.iterrows():
    buffer = site.geometry.buffer(constants.SITE_BUFFER)
    for var_idx, ext_file in enumerate(ext_file_list):
        with rasterio.open(ext_file, "r+") as src:
            out_image, out_transform = rasterio.mask.mask(src, [mapping(buffer)], crop=True)
            out_image = np.ma.masked_where((out_image == -9999)|(out_image == 9999), out_image)
            pit_ext.iat[site_idx, pit_ext.columns.get_loc(ext_file_var[var_idx])] = out_image[0]

In [ ]:
# Extract the veg and dem metrics
# TODO: Add masked gradient for slope
aux_stats = pd.DataFrame()
for idx, pit in pit_ext.iterrows():
    veg_cover = [0] * len(constants.VEG_CLASS)
    veg_ext = pit['veg_type'][pit['veg_type'].mask == False].data
    (veg_type, veg_count) = np.unique(veg_ext, return_counts=True)
    veg_per = (veg_count/veg_count.sum())*100
    for i,vtype in enumerate(veg_type):
        veg_cover[vtype] = veg_per[i]

    veg_h_mean = pit['veg_height'].mean()
    veg_h_std = pit['veg_height'].std()

    dem_h_mean = pit['dtm_elv'].mean()
    dem_h_std = pit['dtm_elv'].std()
    
    dem_s_mean = np.mean(pit['dtm_slope'])
    dem_s_std = np.std(pit['dtm_slope'])
    
    aux_stats = pd.concat([aux_stats,pd.DataFrame({'site': [pit['site']],
                                   'pit_ts': [pit['timestamp']],
                                   'veg_c_per': [veg_cover],
                                   'veg_h_mean': [veg_h_mean],
                                   'veg_h_std': [veg_h_std], 
                                   'dem_h_mean': [dem_h_mean],
                                   'dem_h_std': [dem_h_std],
                                   'dem_s_mean': [dem_s_mean],
                                   'dem_s_std': [dem_s_std]})],ignore_index=True)
# Merge the aux and snow stats for each site
pit_stats = pd.merge(aux_stats, snow_stats, on='site')
pit_stats.to_csv("../Data/pit_stats.csv")

In [ ]:
# Get average site composition
cum_cov = [sum(x) for x in zip(*pit_stats['veg_c_per'].values)]
cum_elm = len(pit_stats['veg_c_per'])
avg_cov = [i / cum_elm for i in cum_cov]
print(constants.VEG_CLASS)
print(avg_cov)

In [ ]:
# Example of the extracted data
site_name = 'RS26'
site_analysis = pit_ext[pit_ext['site'].str.contains(site_name)]
site_aux = aux_stats[aux_stats['site'].str.contains(site_name)]
site_snow = snow_stats[snow_stats['site'].str.contains(site_name)]
print('Site {}'.format(site_name))
print('Mean MP Snow (cm): {:.3}'.format(site_snow['depth_mean'].values[0]))
print('Weighted Cutter Dens.: {:.4}'.format(site_snow['dens_mean'].values[0]))
print('Mean SWE (mm): {:.4}'.format(site_snow['swe_mean'].values[0]))
print('Mean IC SSA: {:.3}'.format(site_snow['ssa_mean'].values[0]))
print('Mean Veg. Height: {:.2} m\n'.format(site_aux['veg_h_mean'].values[0]))
for vi, vc in enumerate(constants.VEG_CLASS):
    print('{}: {}%'.format(vc, np.round(pit_stats['veg_c_per'].values[0][vi],1)))

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 10))
ax1.imshow(site_analysis['veg_type'].values[0])
ax2.imshow(np.log(site_analysis['veg_height'].values[0]* 200))
ax3.imshow(site_analysis['dtm_elv'].values[0])

ax1.set_title('Veg. Type')
ax2.set_title('Veg. Height')
ax3.set_title('Elevation')
fig.tight_layout()

Site RS26  
Mean MP Snow (cm): 41.8  
Weighted Cutter Dens.: 227.7  
Mean SWE (mm): 95.11  
Mean IC SSA: 15.1  
Mean Veg. Height: 0.03 m  

Tree: 0%  
Tall Shrub: 0%  
Riparian Shrub: 0%  
Dwarf Shrub: 0%  
Tussock: 0%  
Lichen: 100.0%  
Water: 0%  


<center><img src="../Figures/Part_1_ETL_Field_Fig4.png" height="500px"></center>

<center>Figure: Example of vegetation type, height and DSM elevation for site RS26</center>

# Save the processed field datasets

In [ ]:
site_meta.to_pickle("../Data/site_meta.pkl")
ref_ssa.to_pickle("../Data/ref_ssa.pkl")
ref_rho.to_pickle("../Data/ref_rho.pkl")
ref_temp.to_pickle("../Data/ref_temp.pkl")
mp_utm.to_pickle("../Data/ref_mp.pkl")
pit_meta.to_pickle("../Data/ref_pit.pkl")
pit_stats.to_pickle("../Data/pit_stats.pkl")
pit_ext.to_pickle("../Data/pit_aux.pkl")